---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [183]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [184]:
pd.options.display.float_format = '{:.3f}'.format # suppress scientific notation

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [185]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [186]:
# load City_Zhvi_AllHomes.csv
median_home_sales_price = pd.read_csv('City_Zhvi_AllHomes.csv')

# map the state name based on the abbreviation
median_home_sales_price['StateName'] = median_home_sales_price['State'].map(states)
median_home_sales_price['StateName'].value_counts()

# keep period from Q1/2000 to Q3/2016
columns_to_keep = list(range(1,2)) + list(range(51, len(median_home_sales_price.columns)))
median_home_sales_price = median_home_sales_price.iloc[:, columns_to_keep]

median_home_sales_price.head()
# median_home_sales_price.info() # (10730, 252)

,RegionName,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,...,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,StateName
0,New York,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,576200,578400,582200,588000,592200,592500,590200,588000,586400,New York
1,Los Angeles,204400.000,207000.000,209800.000,212300.000,214500.000,216600.000,219000.000,221100.000,222800.000,...,560800,562800,565600,569700,574000,577800,580600,583000,585100,California
2,Chicago,136800.000,138300.000,140100.000,141900.000,143700.000,145300.000,146700.000,147900.000,149000.000,...,206900,206200,205800,206200,207300,208200,209100,211000,213000,Illinois
3,Philadelphia,52700.000,53100.000,53200.000,53400.000,53700.000,53800.000,53800.000,54100.000,54500.000,...,121600,121800,123300,125200,126400,127000,127400,128300,129100,Pennsylvania
4,Phoenix,111000.000,111700.000,112800.000,113700.000,114300.000,115100.000,115600.000,115900.000,116500.000,...,185300,186600,188000,189100,190200,191300,192800,194500,195900,Arizona


In [187]:
# load university_towns.txt
university_towns = pd.read_csv('university_towns.txt',
                               header = None,
                               delimiter = "\n",
                               usecols = [0],
                               names = ['RegionName'])

# country cleanup
university_towns.insert(0, 'State', university_towns['RegionName'].str.extract('(.*)\[edit\]', expand=False).ffill())
university_towns['RegionName'] = university_towns['RegionName'].str.replace(r' \(.+$', '')
university_towns = university_towns[~university_towns['RegionName'].str.contains('\[edit\]')].reset_index(drop=True)

university_towns.head()
# university_towns.info() # (517, 2)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [189]:
# load gdplev.xls
gdp_data = pd.read_excel('gdplev.xls',
                         skiprows = 7,
                         usecols = [4,5,6],
                         names = ['Period', 'GDP_USD_current', 'GDP_USD_chained'])

# subsetting to period of Q1 2000 onward
gdp_data = gdp_data[gdp_data['Period'] >= '2000q1']

# set the index to period
gdp_data = gdp_data.set_index('Period')

gdp_data.head()
# gdp_data.info() # (65,3)

,GDP_USD_current,GDP_USD_chained
Period,,
2000q1,10031.000,12359.100
2000q2,10278.300,12592.500
2000q3,10357.400,12607.700
2000q4,10472.300,12679.300
2001q1,10508.100,12643.300


In [192]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return university_towns

# get_list_of_university_towns()
# type(get_list_of_university_towns())
# pandas.core.frame.DataFrame

In [193]:
# find out the starting and ending period of recession
recession_start = list()
recession_end = list()
qtr_before_recession = list()
recession = False

for i in range(1, len(gdp_data)-1):
    if recession == False and (gdp_data['GDP_USD_chained'].iloc[i-1] > gdp_data['GDP_USD_chained'].iloc[i] > gdp_data['GDP_USD_chained'].iloc[i+1]):
        recession = True
        recession_start.append(gdp_data.index[i])
        qtr_before_recession.append(gdp_data.index[i-1])
    elif recession ==  True and (gdp_data['GDP_USD_chained'].iloc[i-1] < gdp_data['GDP_USD_chained'].iloc[i] < gdp_data['GDP_USD_chained'].iloc[i+1]):
        recession = False
        recession_end.append(gdp_data.index[i+1])

print('Starting period of recession:', recession_start[0])
print('Ending period of recession:', recession_end[0])
print('Quarter before recession starts:', qtr_before_recession[0])

Starting period of recession: 2008q3
Ending period of recession: 2009q4
Quarter before recession starts: 2008q2


In [196]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return recession_start[0]

# get_recession_start() # '2008q3'
# type(get_recession_start())

str

In [200]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return recession_end[0]

# get_recession_end() # '2009q4'
# type(get_recession_end())

str

In [203]:
# subset to recession period only
gdp_data_recession = gdp_data[(gdp_data.index.values >= recession_start[0]) & (gdp_data.index.values <= recession_end[0])]
recession_bottom = gdp_data_recession['GDP_USD_chained'].idxmin()

def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return recession_bottom

# get_recession_bottom() # '2009q2'
# type(get_recession_bottom())

str

In [209]:
# group the columns to quarters and calculate the mean values
median_home_sales_price_indexed = median_home_sales_price.set_index(['StateName', 'RegionName'])
median_home_sales_price_indexed.index.names = (['State', 'RegionName'])

housing_data_to_quarters = (median_home_sales_price_indexed
                            .groupby(pd.PeriodIndex(median_home_sales_price_indexed.columns, freq='Q'), axis=1)
                            .mean()
                            .rename(columns=lambda c: str(c).lower()))

housing_data_to_quarters.head()
# housing_data_to_quarters.shape # (10730, 67)

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,515466.667,522800.000,528066.667,532266.667,540800.000,557200.000,572833.333,582866.667,591633.333,587200.000
California,Los Angeles,207066.667,214466.667,220966.667,226166.667,233000.000,239100.000,245066.667,253033.333,261966.667,272700.000,...,498033.333,509066.667,518866.667,528800.000,538166.667,547266.667,557733.333,566033.333,577466.667,584050.000
Illinois,Chicago,138400.000,143633.333,147866.667,152133.333,156933.333,161800.000,166400.000,170433.333,175500.000,177566.667,...,192633.333,195766.667,201266.667,201066.667,206033.333,208300.000,207900.000,206066.667,208200.000,212000.000
Pennsylvania,Philadelphia,53000.000,53633.333,54133.333,54700.000,55333.333,55533.333,56266.667,57533.333,59133.333,60733.333,...,113733.333,115300.000,115666.667,116200.000,117966.667,121233.333,122200.000,123433.333,126933.333,128700.000
Arizona,Phoenix,111833.333,114366.667,116000.000,117400.000,119600.000,121566.667,122700.000,124300.000,126533.333,128366.667,...,164266.667,165366.667,168500.000,171533.333,174166.667,179066.667,183833.333,187900.000,191433.333,195200.000


In [210]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return housing_data_to_quarters

# convert_housing_data_to_quarters()
# type(convert_housing_data_to_quarters())
# convert_housing_data_to_quarters().shape # (10730, 67)

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,515466.667,522800.000,528066.667,532266.667,540800.000,557200.000,572833.333,582866.667,591633.333,587200.000
California,Los Angeles,207066.667,214466.667,220966.667,226166.667,233000.000,239100.000,245066.667,253033.333,261966.667,272700.000,...,498033.333,509066.667,518866.667,528800.000,538166.667,547266.667,557733.333,566033.333,577466.667,584050.000
Illinois,Chicago,138400.000,143633.333,147866.667,152133.333,156933.333,161800.000,166400.000,170433.333,175500.000,177566.667,...,192633.333,195766.667,201266.667,201066.667,206033.333,208300.000,207900.000,206066.667,208200.000,212000.000
Pennsylvania,Philadelphia,53000.000,53633.333,54133.333,54700.000,55333.333,55533.333,56266.667,57533.333,59133.333,60733.333,...,113733.333,115300.000,115666.667,116200.000,117966.667,121233.333,122200.000,123433.333,126933.333,128700.000
Arizona,Phoenix,111833.333,114366.667,116000.000,117400.000,119600.000,121566.667,122700.000,124300.000,126533.333,128366.667,...,164266.667,165366.667,168500.000,171533.333,174166.667,179066.667,183833.333,187900.000,191433.333,195200.000
Nevada,Las Vegas,132600.000,134366.667,135400.000,137000.000,139533.333,141733.333,143366.667,146133.333,149333.333,150933.333,...,170066.667,173400.000,175466.667,177500.000,181600.000,186766.667,190633.333,194600.000,197200.000,199950.000
California,San Diego,222900.000,234366.667,245433.333,256033.333,267200.000,276266.667,284500.000,291933.333,301233.333,312866.667,...,480200.000,489033.333,496433.333,503366.667,512066.667,519766.667,525466.667,529333.333,536233.333,539750.000
Texas,Dallas,84466.667,83866.667,84866.667,87833.333,89733.333,89300.000,89066.667,90900.000,92566.667,93800.000,...,106633.333,108900.000,111533.333,113700.000,121133.333,128566.667,134600.000,140500.000,144600.000,149300.000
California,San Jose,374266.667,406566.667,431866.667,455500.000,470666.667,470200.000,456800.000,445566.667,441433.333,457766.667,...,679400.000,697033.333,714933.333,731433.333,756733.333,776400.000,789133.333,803600.000,818933.333,822200.000


In [211]:
# price_ratio = qtr_before recession_start / recession_bottom
housing_data_to_quarters_copy = housing_data_to_quarters.copy()
housing_data_to_quarters_copy['Price_ratio'] = housing_data_to_quarters_copy[qtr_before_recession].div(housing_data_to_quarters_copy[recession_bottom], axis = 0)
housing_data_to_quarters_copy['Price_ratio'].head()

State         RegionName  
New York      New York       1.082
California    Los Angeles    1.213
Illinois      Chicago        1.083
Pennsylvania  Philadelphia   1.017
Arizona       Phoenix        1.220
Name: Price_ratio, dtype: float64

In [212]:
# divide the dataframe to two groups: university and non-university towns
university_towns_list = university_towns.to_records(index = False).tolist()
university_towns_grp = housing_data_to_quarters_copy.loc[housing_data_to_quarters_copy.index.isin(university_towns_list)]
non_university_towns_grp = housing_data_to_quarters_copy.loc[-housing_data_to_quarters_copy.index.isin(university_towns_list)]

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


In [213]:
# calculate the t-test for 2 independent samples assuming equal variances in samples
ttest_result = ttest_ind(university_towns_grp['Price_ratio'], non_university_towns_grp['Price_ratio'], nan_policy='omit')
print('T-test statistic:', ttest_result[0])
print('T-test p-value:', ttest_result[1])

T-test statistic: -2.99803266418
T-test p-value: 0.00272406370475


In [214]:
# prepare the output - 1)different
def ttest_different():
    
    if ttest_result[1] < 0.01:
        return True
    else:
        return False
    
# ttest_different()
ttest_different = ttest_different()

In [215]:
# prepare the output - 2)p-value
def ttest_pvalue():
    return ttest_result[1]

# ttest_pvalue() 
ttest_pvalue = ttest_pvalue()

In [216]:
# prepare the output - 3)better
def ttest_better():
    
    if university_towns_grp['Price_ratio'].mean() <= non_university_towns_grp['Price_ratio'].mean():
        return 'university town'
    else:
        return 'non-university town'
    
# ttest_better()
ttest_better = ttest_better()

In [217]:
# university_towns_grp['Price_ratio'].mean() # 1.0545645283851421
# non_university_towns_grp['Price_ratio'].mean() #1.0751962209307955

In [220]:
ttest_tuple = ()

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return ttest_tuple + (ttest_different, ttest_pvalue, ttest_better)

# run_ttest() # (True, 0.0027240637047531249, 'university town')
# type(run_ttest()) # tuple

tuple